In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!curl -fsSL https://ollama.com/install.sh | sh
!pip install -q rank_bm25 faiss-cpu sentence-transformers transformers accelerate peft bitsandbytes datasets scikit-learn langchain streamlit

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%                                 13.4%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 55.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 15.8 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 91.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 91.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1) Install dependencies

In [3]:
import subprocess, time, os, sys, json

# Start the Ollama server (daemon)
ollama_proc = subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
time.sleep(3)
print("✅ Ollama server starting...")

# pull a small, reliable model for CPU inference
!ollama pull mistral


✅ Ollama server starting...
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling f5074b1221da:   1% ▕                  ▏  36 MB/4.4 GB                  pulling manifest 
pulling f5074b1221da:   2% ▕                  ▏  99 MB/4.4 GB                  pulling manifest 
pulling f5074b1221da:   3% ▕                  ▏ 131 MB/4.4 GB                  pulling manifest 
pulling f5074b1221da:   4% ▕                  ▏ 192 MB/4.4 GB                  pulling manifest 
pulling f5074b1221da:   6% ▕█                 ▏ 249 MB/4.4 GB                  pulling manifest 
pulling f5074b1221da:   6% ▕█                 ▏ 275 MB/4.4 GB                  pulling manifest 
pulling f5074b1221da:   8% ▕█                 ▏ 337 MB/4.4 GB                  pulling manifest 
pulling f5074b1221da:   9% ▕

2) Start Ollama & pull a small LLM (Mistral)

In [4]:
import os, pandas as pd

INPUT_PATH = "/kaggle/input"
DEFAULT_CORPUS = "/kaggle/working/domain_corpus.csv"

# Try to find a corpus in /kaggle/input; else create a small toy set
found = None
for root, _, files in os.walk(INPUT_PATH):
    for f in files:
        if f.lower() == "domain_corpus.csv":
            found = os.path.join(root, f)
            break
    if found: break

if not found:
    data = [
        {"doc_id": "D1", "title": "EV Market in India", "text": "Tata Motors leads EV passenger segment in India. Sales grew in 2024 driven by tax incentives and charging infra expansion."},
        {"doc_id": "D2", "title": "Automotive Quarterly", "text": "Maruti, Tata, and Mahindra posted mixed results. Commodity prices affected margins; exports to Africa increased."},
        {"doc_id": "D3", "title": "Finance News", "text": "Tata Motors stock showed volatility with quarterly earnings beats but margin concerns. Analysts debated valuation multiples."},
        {"doc_id": "D4", "title": "Battery Tech", "text": "LFP and NMC chemistries dominate; solid-state remains in R&D. Suppliers invest in recycling and second-life applications."},
    ]
    pd.DataFrame(data).to_csv(DEFAULT_CORPUS, index=False)
    CORPUS_PATH = DEFAULT_CORPUS
else:
    CORPUS_PATH = found

print("Using corpus:", CORPUS_PATH)
df = pd.read_csv(CORPUS_PATH)
df.head()


Using corpus: /kaggle/working/domain_corpus.csv


,doc_id,title,text
0,D1,EV Market in India,Tata Motors leads EV passenger segment in Indi...
1,D2,Automotive Quarterly,"Maruti, Tata, and Mahindra posted mixed result..."
2,D3,Finance News,Tata Motors stock showed volatility with quart...
3,D4,Battery Tech,LFP and NMC chemistries dominate; solid-state ...


3) Create / Load a domain corpus

Provide a CSV named domain_corpus.csv with columns: doc_id,title,text.
Below, we auto-create a tiny toy corpus if none is attached.

In [5]:
from typing import List, Dict
import math

def chunk_text(text: str, chunk_size: int = 400, overlap: int = 80) -> List[str]:
    words = text.split()
    chunks = []
    i = 0
    while i < len(words):
        chunk = words[i:i+chunk_size]
        chunks.append(" ".join(chunk))
        i += (chunk_size - overlap)
    return chunks

# Build passage list with metadata
passages = []
for _, row in df.iterrows():
    psgs = chunk_text(str(row["text"]))
    for j, p in enumerate(psgs):
        passages.append({
            "doc_id": row["doc_id"],
            "title": row.get("title", ""),
            "pid": f'{row["doc_id"]}_p{j}',
            "text": p
        })

len(passages), passages[:2]


(4,
 [{'doc_id': 'D1',
   'title': 'EV Market in India',
   'pid': 'D1_p0',
   'text': 'Tata Motors leads EV passenger segment in India. Sales grew in 2024 driven by tax incentives and charging infra expansion.'},
  {'doc_id': 'D2',
   'title': 'Automotive Quarterly',
   'pid': 'D2_p0',
   'text': 'Maruti, Tata, and Mahindra posted mixed results. Commodity prices affected margins; exports to Africa increased.'}])

4) Chunking into passages

In [6]:
from rank_bm25 import BM25Okapi

bm25_corpus = [p["text"] for p in passages]
bm25 = BM25Okapi([t.split() for t in bm25_corpus])

def bm25_retrieve(query: str, topk: int = 20) -> List[int]:
    scores = bm25.get_scores(query.split())
    # return indices of topk passages
    idxs = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:topk]
    return idxs


5) BM25 retriever

6) Bi-encoder (dense) retriever with FAISS

In [7]:
import numpy as np, faiss
from sentence_transformers import SentenceTransformer

bi_model_name = "sentence-transformers/all-MiniLM-L6-v2"
biencoder = SentenceTransformer(bi_model_name)

passage_embeddings = biencoder.encode([p["text"] for p in passages], show_progress_bar=False, convert_to_numpy=True, normalize_embeddings=True)
dim = passage_embeddings.shape[1]
index = faiss.IndexFlatIP(dim)  # cosine via normalized dot
index.add(passage_embeddings)

def bi_retrieve(query: str, topk: int = 20) -> List[int]:
    q_emb = biencoder.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    D, I = index.search(q_emb, topk)
    return I[0].tolist()


2025-08-15 06:31:03.947327: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755239464.195481      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755239464.264480      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

7) Cross-encoder reranker

In [8]:
from sentence_transformers import CrossEncoder

# a small cross-encoder that works well for reranking
cross_model_name = "cross-encoder/ms-marco-MiniLM-L-6-v2"
cross_encoder = CrossEncoder(cross_model_name)

def cross_rerank(query: str, cand_indices: List[int], topk: int = 5) -> List[int]:
    pairs = [[query, passages[i]["text"]] for i in cand_indices]
    scores = cross_encoder.predict(pairs)
    ranked = sorted(zip(cand_indices, scores), key=lambda x: x[1], reverse=True)
    return [i for i, _ in ranked[:topk]]


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

8) Hybrid retrieval (BM25 + Bi-encoder → Cross-encoder rerank)

In [9]:
def hybrid_retrieve(query: str, bm25_k: int = 30, bi_k: int = 30, final_k: int = 5) -> List[Dict]:
    idx_bm25 = bm25_retrieve(query, topk=bm25_k)
    idx_bi = bi_retrieve(query, topk=bi_k)
    # merge unique
    merged = list(dict.fromkeys(idx_bm25 + idx_bi))
    if not merged:
        return []
    final_idxs = cross_rerank(query, merged, topk=final_k)
    return [passages[i] for i in final_idxs]


In [15]:
import sys, subprocess, pkgutil

def pip_install(*pkgs):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", *pkgs])

# Core + integrations (version-pinned for compatibility)
pip_install(
    "langchain==0.2.*",
    "langchain-community==0.2.*",
    "langchain-core==0.2.*",
    "langchain-text-splitters==0.2.*",
    "faiss-cpu",
    "sentence-transformers",
    "rank-bm25>=0.2.2",
    "transformers",
    "accelerate",
    "peft",
    "bitsandbytes",
    "datasets",
    "scikit-learn",
    "streamlit",
    "ollama"  # python client (nice to have)
)

# quick sanity check
print("langchain_community found?",
      pkgutil.find_loader("langchain_community") is not None)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.1/397.1 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.4/483.4 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.9/503.9 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
   

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.7.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.1 which is incompatible.


langchain_community found? True


In [16]:
from langchain_community.llms import Ollama
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

print("Imports OK ✅")


Imports OK ✅


9) RAG prompt assembly + call Ollama (Mistral)

In [17]:
# from langchain.llms import Ollama
from langchain_community.llms import Ollama
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings


llm = Ollama(model="mistral", base_url="http://127.0.0.1:11434")

def build_prompt(query: str, contexts: List[Dict]) -> str:
    context_block = "\n\n".join([f"[{c['doc_id']} | {c['title']} | {c['pid']}]\n{c['text']}" for c in contexts])
    instr = (
        "You are a careful assistant. Answer ONLY using the provided context. "
        "If the answer is not in the context, say 'I don’t know' and do not fabricate."
    )
    return f"""{instr}

Context:
{context_block}

Question: {query}

Answer:"""

def rag_answer(query: str, topk: int = 5) -> Dict:
    ctx = hybrid_retrieve(query, final_k=topk)
    prompt = build_prompt(query, ctx)
    ans = llm(prompt)
    return {"answer": ans, "contexts": ctx}

# quick smoke test
print(rag_answer("Which company leads EV passenger segment in India?", topk=3)["answer"])


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/tmp/ipykernel_36/3090020118.py:27: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ans = llm(prompt)


 Tata Motors


10) Evaluation (Exact Match)

Create or attach a dev set CSV dev_qa.csv with columns: question,answer.
We’ll auto-create a tiny toy set if none exists.

In [18]:
DEV_PATH = "/kaggle/working/dev_qa.csv"
if not os.path.exists(DEV_PATH):
    pd.DataFrame([
        {"question": "Who leads EV passenger segment in India?", "answer": "Tata Motors"},
        {"question": "Which chemistries dominate batteries mentioned?", "answer": "LFP and NMC"},
        {"question": "Did analysts debate valuation multiples for Tata Motors?", "answer": "Yes"},
    ]).to_csv(DEV_PATH, index=False)

dev = pd.read_csv(DEV_PATH)
dev.head()


,question,answer
0,Who leads EV passenger segment in India?,Tata Motors
1,Which chemistries dominate batteries mentioned?,LFP and NMC
2,Did analysts debate valuation multiples for Ta...,Yes


In [19]:
import re

def normalize(s: str) -> str:
    s = s or ""
    s = s.strip().lower()
    s = re.sub(r"\s+", " ", s)
    return s

def exact_match(pred: str, gold: str) -> int:
    return int(normalize(pred) == normalize(gold))

def evaluate_em(rows: pd.DataFrame, topk: int = 5) -> float:
    hits = 0
    for _, r in rows.iterrows():
        out = rag_answer(r["question"], topk=topk)
        pred = out["answer"]
        # crude: take first line
        pred_line = (pred.splitlines()+[""])[0]
        hits += exact_match(pred_line, r["answer"])
    return hits / len(rows)

em = evaluate_em(dev, topk=5)
print(f"Exact Match (EM): {em:.3f}")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Exact Match (EM): 0.333


11) Switchable retriever ablations (BM25 / Bi-encoder / Hybrid)

In [20]:
def retrieve_variant(query: str, mode: str = "hybrid", topk: int = 5) -> List[Dict]:
    if mode == "bm25":
        idxs = bm25_retrieve(query, topk=topk)
        return [passages[i] for i in idxs]
    elif mode == "bi":
        idxs = bi_retrieve(query, topk=topk)
        return [passages[i] for i in idxs]
    elif mode == "hybrid":
        return hybrid_retrieve(query, final_k=topk)
    else:
        raise ValueError("mode must be one of {'bm25','bi','hybrid'}")

def rag_answer_with_mode(query: str, mode: str = "hybrid", topk: int = 5) -> Dict:
    ctx = retrieve_variant(query, mode=mode, topk=topk)
    prompt = build_prompt(query, ctx)
    ans = llm(prompt)
    return {"answer": ans, "contexts": ctx}

def evaluate_em_mode(rows: pd.DataFrame, mode: str, topk: int = 5) -> float:
    hits = 0
    for _, r in rows.iterrows():
        out = rag_answer_with_mode(r["question"], mode=mode, topk=topk)
        pred = (out["answer"].splitlines()+[""])[0]
        hits += exact_match(pred, r["answer"])
    return hits / len(rows)

for m in ["bm25","bi","hybrid"]:
    score = evaluate_em_mode(dev, mode=m, topk=5)
    print(f"{m.upper()} EM = {score:.3f}")


BM25 EM = 0.333


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

BI EM = 0.000


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

HYBRID EM = 0.333


12) Optional: LoRA / QLoRA fine-tuning stubs (OFF by default)

Training large LLMs on Kaggle CPU is impractical.
The cell below shows how you would LoRA/QLoRA-tune a small model (e.g., TinyLlama/TinyLlama-1.1B-Chat-v1.0) if you switch to a GPU runtime and set ENABLE_TRAINING=True.
This is illustrative code aligning with your spec; keep it disabled on CPU.

In [21]:
ENABLE_TRAINING = False  # set True only with GPU runtime (and enough RAM)

if ENABLE_TRAINING:
    import torch
    from datasets import Dataset
    from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, DataCollatorForLanguageModeling, Trainer
    from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

    base_model = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # ~1.1B params; still heavy
    tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
    model = AutoModelForCausalLM.from_pretrained(
        base_model,
        device_map="auto",
        load_in_8bit=True,          # QLoRA-ish
        torch_dtype=torch.float16
    )
    model = prepare_model_for_kbit_training(model)

    peft_cfg = LoraConfig(
        r=8, lora_alpha=16, lora_dropout=0.05,
        bias="none", task_type="CAUSAL_LM"
    )
    model = get_peft_model(model, peft_cfg)

    # Build a toy instruction dataset from our corpus (question-context-answer)
    # In real use, load your labeled domain QA pairs.
    def build_pairs(df):
        rows = []
        for _, r in dev.iterrows():
            ctx = hybrid_retrieve(r["question"], final_k=3)
            ctx_text = "\n\n".join([c["text"] for c in ctx])
            prompt = f"Context:\n{ctx_text}\n\nQuestion: {r['question']}\nAnswer:"
            target = r["answer"]
            rows.append({"text": f"{prompt} {target}"})
        return rows

    train_rows = build_pairs(df)
    ds = Dataset.from_pandas(pd.DataFrame(train_rows))

    def tokenize_fn(examples):
        return tokenizer(examples["text"], truncation=True, max_length=512)

    tok = ds.map(tokenize_fn, batched=True, remove_columns=["text"])
    collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

    args = TrainingArguments(
        output_dir="/kaggle/working/lora-out",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        num_train_epochs=1,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        save_steps=1000,
        optim="paged_adamw_8bit"  # for QLoRA
    )

    trainer = Trainer(model=model, args=args, train_dataset=tok, data_collator=collator)
    trainer.train()

    # Save LoRA adapters
    model.save_pretrained("/kaggle/working/lora-out")
    tokenizer.save_pretrained("/kaggle/working/lora-out")

    print("✅ LoRA/QLoRA adapters saved at /kaggle/working/lora-out")



13) Usage examples

In [22]:
print("=== Hybrid RAG ===")
out = rag_answer("What chemistries dominate batteries?", topk=3)
print(out["answer"])
print("\nContexts used:")
for c in out["contexts"]:
    print(f"- {c['doc_id']} | {c['title']} | {c['pid']}")


=== Hybrid RAG ===


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 LFP and NMC chemistries dominate.

Contexts used:
- D4 | Battery Tech | D4_p0
- D4 | Battery Tech | D4_p0
- D1 | EV Market in India | D1_p0
